In [ ]:
# Dashboard
import plotly
import plotly.express as px
import plotly.graph_objects as go

import dash
import dash_core_components as dcc
import dash_html_components as html
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output

# Classicals
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn.model_selection import cross_validate

In [ ]:
# Preperation
df_1 = pd.read_csv('dashboard_airlinequality.csv')
df_2 = pd.read_csv('dashboard_tab2.csv')
cols_15 = ['Seat Comfort', 'Cabin Staff Service', 'Ground Service', 
            'Value For Money', 'Food & Beverages', 'Inflight Entertainment']
li_seat_type = ['Economy Class', 'Business Class', 'Premium Economy', 'First Class']
li_type_of_traveller = ['Solo Leisure', 'Business', 'Family Leisure', 'Couple Leisure']
li_flight_length = ['Short-Haul', 'Middle-Haul', 'Long-Haul']

## Tab1: What contribute to customer satisfaction?

In [ ]:
# A function to output the chart
def output_chart_1(df = df_1,
                 seat_type = ['Economy Class'], 
                 type_of_travaller = ['Solo Leisure'], 
                 stop = [False],
                 flight_length = ['Short-Haul']):
    
    # Select data by category
    mask = (df['Seat Type'].isin(seat_type)) &\
            (df['Type Of Traveller'].isin(type_of_travaller)) &\
            (df['Is_stop'].isin(stop)) &\
            (df['Flight Length'].isin(flight_length))
    df = df[mask]
    
    # Fill nans with mean
    df = df.fillna(df.mean())
            
    # Make X and y
    X = df[cols_15]
    y = df['Recommended']
    
    # Calculate coefficients
    model = linear_model.LogisticRegression(solver='lbfgs', C=0.05)
    model.fit(X, y)
    df_coef = pd.DataFrame({'Feature':X.columns.to_list(),
                        'Coef':model.coef_.tolist()[0]})
    df_coef = df_coef.sort_values(['Coef'])
    df_coef['Importance'] = np.exp(df_coef['Coef']) - 1
    
    # Calculate some statistics
    model_accuracy = cross_validate(model, X, y, cv=5, return_train_score=True)['test_score'].mean()
    n_samples = X.shape[0]
    print(model_accuracy, n_samples)
    
    # Visualize coefficients
    # Intepretation: If the coefficient for Inflight Entertainment is 0.2,
    #                an increase in a star in Inflight Entertainment will make a customer 22% (exp(0.2)=1.22) 
    #                more likely to recommend the flight 
    fig = px.bar(df_coef, y='Feature', x='Importance', color='Feature', orientation='h')
    
    fig.update_layout(title='What Contribute to Customer Satisfaction',
                      yaxis_title='Aspects',
                      xaxis_title='An increase in 1 star makes a customer X% more likely to recommend a flight',
                      xaxis_tickformat = ',.0%',
                      showlegend=False,
                      font={'size':16})
    
    return fig
    

In [4]:
# Layout
tab1 = html.Div(
    dbc.Row([
        dbc.Col([
                html.H6('Choose your viz'),
            
                html.Label('Class'),
                dcc.Dropdown(
                id='class_tab1', 
                options=[{'label': i, 'value': i} for i in li_seat_type],
                multi=True,
                value=['Economy Class']), 

                html.Label('Travel Purpose'),
                dcc.Dropdown(
                id='purpose_tab1',    
                options=[{'label': i, 'value': i} for i in li_type_of_traveller],
                multi=True,
                value=['Solo Leisure']),

                html.Label('Flight Length'),
                dcc.Dropdown(
                id='length_tab1',    
                options=[{'label': i.replace('-',' '), 'value': i} for i in li_flight_length],
                multi=True,
                value=['Short-Haul']),

                html.Label('Is there a stop'),
                dcc.Dropdown(
                id='stop_tab1',    
                options=[{'label': 'Non-stop', 'value': True},
                         {'label': 'Stop', 'value': False}],
                multi=True,
                value=[True])],md=4),
        
        dbc.Col([dcc.Graph(id='chart_tab1')])
    
    ]))

## Tab-2: What do customers make of the flights

In [5]:
def output_chart_2(df = df_2,
                 choice_class = ['Economy Class'],
                 choice_flight_length = ['Short-Haul']):
    
    # Select data by category
    mask = df['Class'].isin(choice_class) &\
    df['Flight Length'].isin(choice_flight_length)
    df = df[mask]
    
    # Calculate the mean ratings
    cols_ratings = ['Cabin Staff Service', 'Food & Beverages','Ground Service', 
                    'Inflight Entertainment', 'Seat Comfort', 'Value for Money']
    df_for_viz = df[cols_ratings].mean().reset_index()

    # Calculate some statistics
    n_samples = df.shape[0]
    print(n_samples)
    
    # Visualize ratings
    fig = px.line_polar(df_for_viz, r=0, theta='index', color='',line_close=True)
    fig.update_traces(fill='toself')
    fig.update_layout(
      polar=dict(
        radialaxis=dict(
          visible=True,
          range=[0, 5]
        )),
      showlegend=False
    )
    fig.update_layout(title='What do Customers Make of the Flights',
                      title_x=0.5,
                      font={'size':16})
    
    return fig


In [6]:
# Layout
tab2 = dbc.Container([
    dbc.Row([
            html.Div([
            html.Label('Class'),
            dcc.Dropdown(
            id='class_tab2', 
            options=[{'label': i, 'value': i} for i in li_seat_type],
            multi=True,
            value=['Economy Class'])]), 
            
            html.Div([
            html.Label('Flight Length'),
            dcc.Dropdown(
            id='length_tab2',    
            options=[{'label': i.replace('-',' '), 'value': i} for i in li_flight_length],
            multi=True,
            value=['Short-Haul'])])]),
    
    dbc.Row([dbc.Col(dcc.Graph(id='chart_cloud_tab2')),
            dbc.Col(dcc.Graph(id='chart_spider_tab2'))])
    
])

In [ ]:
# Running the dashboard

external_stylesheets = [dbc.themes.BOOTSTRAP]

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div([
    dcc.Tabs(
        [dcc.Tab(label='What Contribute to Customer Satisfaction?', children=tab1),
         dcc.Tab(label='What do Customers Make of the Flights?', children=tab2)]
    )])

# Tab 1 interactivity
@app.callback(
    Output('chart_tab1', 'figure'),
    [Input('class_tab1', 'value'),
     Input('purpose_tab1', 'value'),
     Input('length_tab1', 'value'),
     Input('stop_tab1', 'value')])
def update_figure(seat_type, type_of_travaller, flight_length, stop):
    return output_chart_1(df_1, seat_type, type_of_travaller, stop, flight_length)

# Tab 2 interactivity
@app.callback(
    Output('chart_spider_tab2', 'figure'),
    [Input('class_tab2', 'value'),
     Input('length_tab2', 'value')])
def update_figure(choice_class, choice_flight_length):
    return output_chart_2(df_2, choice_class, choice_flight_length)

if __name__ == '__main__':
    app.run_server(debug=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [29/Jan/2020 17:00:49] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/Jan/2020 17:00:49] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [29/Jan/2020 17:00:49] "GET /_dash-dependencies HTTP/1.1" 200 -


8725
0.9255555555555557 96


127.0.0.1 - - [29/Jan/2020 17:00:51] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [29/Jan/2020 17:00:51] "POST /_dash-update-component HTTP/1.1" 200 -


23612


...

127.0.0.1 - - [29/Jan/2020 17:02:34] "POST /_dash-update-component HTTP/1.1" 200 -


0


127.0.0.1 - - [29/Jan/2020 17:02:36] "POST /_dash-update-component HTTP/1.1" 200 -


674


127.0.0.1 - - [29/Jan/2020 17:02:37] "POST /_dash-update-component HTTP/1.1" 200 -
